In [525]:
import cellgrid 
import numpy as np
from MDAnalysis.lib.distances import distance_array
from scipy.spatial.distance import squareform 

##Initialize the class for benchmarks
class SimpleCGBench():
    """
    Class for simple CG benchmarks
    """
    def __init__(self):
        #self.box = np.array([100.,100.,100.,90.,90.,90.],dtype=np.float32)
        self.box = np.ones(3).astype(np.float32)*100
        self.points = np.random.uniform(low=0,high=1.0,size=(10000,3))*self.box[:3]
        self.points = np.array(self.points,dtype=np.float32)
        self.maxdist = 10.0

    def bf_select(self, box, points, maxdist):
        """ 
        Comparison between cellgrid and brute force method 
        to select all the points within a distance of 
        {maxdist} units using periodic boundary conditions
        Contact maps with single trajectory - Homogeneous points and 
        cutoff radius of 10 units
        Inputs : points, box, maxdist
        Output : Numpy array with point indices and corresponding distance
        """
        distance = self_distance_array(points,box)
        distance = scipy.spatial.distance.squareform(distance)
        distance[np.tril_indices(distance.shape[0])] = (1.1*maxdist)
        mask = np.where((distance < maxdist))
        out = tuple((np.column_stack((mask[0],mask[1])),distance[mask]))
        return out
        
    def cg_select(self, box, points, maxdist):
        indx,dist = cellgrid.capped_self_distance_array(points,maxdist,box=box)
        mask = np.where(dist<maxdist)
        return indx[mask],dist[mask]

In [526]:
ob = SimpleCGBench()
a = ob.cg_select(ob.box,ob.points,ob.maxdist)
type(a)

tuple

In [533]:
p = ob.bf_select(ob.box,ob.points,ob.maxdist)

In [528]:
(a[1].dtype),(p[1].dtype)

(dtype('float32'), dtype('float64'))

In [529]:
np.testing.assert_array_almost_equal(np.sort(a[1].astype(np.float64)),np.sort(p[1]),decimal=5)
np.testing.assert_equal(a[1].shape,p[1].shape)

In [530]:
%timeit ob.bf_select(ob.box,ob.points,ob.maxdist)

8.63 s ± 1.25 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [531]:
%timeit ob.cg_select(ob.box,ob.points,ob.maxdist)

694 ms ± 95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
